In [3]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

os.makedirs("data", exist_ok=True)

API = "2a430097-ac9b-44e4-995e-a02afb9795b2"
SOURCE = "SN68860"
BASE_URL = "https://frost.met.no/observations/v0.jsonld"

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
delta = timedelta(days=30)  # 30-dagers bolker

elements = {
    "air_temperature": "Temperatur (°C)",  
    "mean(air_pressure_at_sea_level P1D)": "Lufttrykk (hPa)",  
    "sum(precipitation_amount P1D)": "Nedbør (mm)"
}

force_update = input("Vil du hente nye data fra Frost API? (ja/nei): ").strip().lower()

def fetch_data():
    filename = os.path.join("data", "frost_data.csv")

    if os.path.exists(filename) and force_update != "ja":
        print("Bruker eksisterende fil:", filename)
        return

    print("Henter data...")

    all_data = pd.DataFrame(columns=["Dato"])

    current_date = start_date
    while current_date <= end_date:
        batch_start = current_date.strftime("%Y-%m-%d")
        batch_end = (current_date + delta).strftime("%Y-%m-%d")
        batch_reference_time = f"{batch_start}/{batch_end}"

        params = {
            "sources": SOURCE,
            "elements": ",".join(elements.keys()),
            "referencetime": batch_reference_time
        }

        response = requests.get(BASE_URL, params=params, auth=(API, ""))
        
        if response.status_code == 200:
            raw_data = response.json().get("data", [])
            batch_df = pd.DataFrame([
                {"Dato": entry["referenceTime"].split("T")[0], 
                 elements[entry["observations"][0]["elementId"]]: entry["observations"][0]["value"]}
                for entry in raw_data
            ])

            # **Løsning på feilen:** Håndter duplikater ved å bruke `suffixes`
            all_data = pd.merge(all_data, batch_df, on="Dato", how="outer", suffixes=("", "_dupe"))

            # Fjern duplikatkolonner hvis de finnes
            for col in elements.values():
                if f"{col}_dupe" in all_data.columns:
                    all_data[col] = all_data[col].combine_first(all_data[f"{col}_dupe"])
                    all_data.drop(columns=[f"{col}_dupe"], inplace=True)

        else:
            print(f"Feil ved henting av data ({batch_reference_time}): {response.status_code}")
            print(response.text)

        current_date += delta

    all_data.to_csv(filename, index=False, encoding="utf-8")
    print("Lagret til:", filename)

fetch_data()


    

Henter data...
Lagret til: data/frost_data.csv


In [5]:
# TEMPFIL 15MIN -> GJENOMSNITT DAG

import pandas as pd
# Leser inn filen med temperaturmålinger
df = pd.read_csv("data/frost_data.csv")

# Beregn gjennomsnittstemperatur per dag
daglig_gjennomsnitt = df.groupby("Dato")["Temperatur (°C)"].mean().reset_index()

# Lagre til ny CSV-fil
daglig_gjennomsnitt.to_csv("data/frost_air_temperature_daglig_gjennomsnitt.csv", index=False)


print("Daglig gjennomsnitt lagret i frost_air_temperature_daglig_gjennomsnitt.csv")


Daglig gjennomsnitt lagret i frost_air_temperature_daglig_gjennomsnitt.csv
